In [34]:
import numpy as np
from scipy import signal

In [50]:
class Layer:
    def __init__(self):
        self.input = None
        self.output = None
    
    def forward(self, input):
        # TODO: return output
        pass

    def backward(self, output_gradient, learning_rate):
        # TODO: update params and return input gradient
        pass

In [51]:
class Dense(Layer):
    def __init__(self, input_size, output_size):
        self.weights = np.random.randn(output_size, input_size)
        self.bias = np.random.randn(output_size, 1)
    
    def forward(self, input):
        self.input = input
        return np.dot(self.weights, self.input) + self.bias
    
    def backward(self, output_gradient, learning_rate):
        # Update params and return input gradient
        weights_gradient = np.dot(output_gradient, self.input.T)
        self.weights -= learning_rate * weights_gradient
        self.bias -= learning_rate * output_gradient
        return np.dot(self.weights.T, output_gradient)

In [52]:
class Activation(Layer):
    def __init__(self, activation, activation_prime):
        self.activation = activation
        self.activation_prime = activation_prime
    
    def forward(self, input):
        self.input = input
        return self.activation(self.input)
    
    def backward(self, output_gradient, learning_rate):
        return np.multiply(output_gradient, self.activation_prime(self.input))

In [53]:
class Tanh(Activation):
    def __init__(self):
        tanh = lambda x: np.tanh(x)
        tanh_prime = lambda x: 1 - np.tanh(x)**2
        super().__init__(tanh, tanh_prime)

In [54]:
class Softmax(Layer):
    def forward(self, input):
        tmp = np.exp(input)
        self.output = tmp / np.sum(tmp)
        return self.output
    
    def backward(self, output_gradient, learning_rate):
        n = np.size(self.output)
        tmp = np.tile(self.output, n)
        return np.dot(tmp * (np.identity(n) - np.transpose(tmp)), output_gradient)

In [55]:
class Convolutional(Layer):
    def __init__(self, input_shape, kernel_size, depth):
        input_depth, input_height, input_width = input_shape
        self.depth = depth # number of kernels
        self.input_shape = input_shape
        self.input_depth = input_depth
        self.output_shape = (depth, input_height - kernel_size + 1, input_width - kernel_size + 1)
        self.kernels_shape = (depth, input_depth, kernel_size, kernel_size) # multiple 3D kernels
        self.kernels = np.random.randn(*self.kernels_shape)
        self.biases = np.random.randn(*self.output_shape)
    

    # https://medium.com/analytics-vidhya/2d-convolution-using-python-numpy-43442ff5f381
    # https://github.com/macbuse/macbuse.github.io/blob/master/PROG/convolve.py
    def convolve2D(self, image: np.ndarray, kernel: np.ndarray, padding = 0, stride = 1):
        # Stride: 1, Padding: 0, Kernel: 3x3, Input: 5x5, Output: 3x3

        # Cross correlation
        kernel = np.flipud(np.fliplr(kernel))

        # Gather shapes of kernel + image + padding
        x_kernel_size, y_kernel_size = kernel.shape
        x_image_size, y_image_size = image.shape[0:2]

        # Pad the image if padding is not 0
        image_padded = np.pad(image, padding, mode='constant', constant_values=(0))

        # Output matrix size: [(n_in - kernel_size + 2*padding) / stride] + 1
        x_output = int(((x_image_size - x_kernel_size + 2 * padding) / stride) + 1)
        y_output = int(((y_image_size - y_kernel_size + 2 * padding) / stride) + 1)
        output = np.zeros((x_output, y_output))

        # Convolution
        for y in range(0, y_image_size - y_kernel_size, stride):
            for x in range(0, x_image_size - x_kernel_size, stride):
                output[x, y] = (kernel * image_padded[x: x + x_kernel_size, y: y + y_kernel_size]).sum()
        
        return output
    
    def forward(self, input):
        self.input = input
        self.output = np.copy(self.biases)
        for i in range(self.depth):
            for j in range(self.input_depth):
                self.output[i] += signal.correlate2d(self.input[j], self.kernels[i][j], "valid")
                # self.output[i] += self.convolve2D(self.input[j], self.kernels[i, j])
        return self.output
    
    def backward(self, output_gradient, learning_rate):
        kernels_gradient = np.zeros(self.kernels_shape)
        input_gradient = np.zeros(self.input_shape)

        for i in range(self.depth):
            for j in range(self.input_depth):
                kernels_gradient[i, j] = signal.correlate2d(self.input[j], output_gradient[i], "valid")
                input_gradient[j] += signal.convolve2d(output_gradient[i], self.kernels[i, j], "full")
        
        self.kernels -= learning_rate * kernels_gradient
        self.biases -= learning_rate * output_gradient
        return input_gradient



In [56]:
class Reshape(Layer):
    def __init__(self, input_shape, output_shape):
        self.input_shape = input_shape
        self.output_shape = output_shape
    
    def forward(self, input):
        return np.reshape(input, self.output_shape)
    
    def backward(self, output_gradient, learning_rate):
        return np.reshape(output_gradient, self.input_shape)

In [57]:
class Sigmoid(Activation):
    def __init__(self):
        def sigmoid(x):
            return 1 / (1 + np.exp(-x))
        
        def sigmoid_prime(x):
            s = sigmoid(x)
            return s * (1 - s)
            
        super().__init__(sigmoid, sigmoid_prime)

In [58]:
def binary_cross_entropy(y_true, y_pred):
    return -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))

def binary_cross_entropy_prime(y_true, y_pred):
    return ((1 - y_true) / (1 - y_pred) - y_true / y_pred) / np.size(y_true)

def mse(y_true, y_pred):
    return np.mean(np.power(y_true - y_pred, 2))

def mse_prime(y_true, y_pred):
    return 2 * (y_pred - y_true) / np.size(y_true)

### XOR-problem

In [59]:
X = np.reshape([[0,0], [0,1], [1,0], [1,1]], (4, 2, 1))
Y = np.reshape([[0], [1], [1], [0]], (4, 1, 1))

In [60]:
network = [
    Dense(2, 3),
    Tanh(),
    Dense(3, 1),
    Tanh()
]

In [61]:
epochs = 10000
learning_rate = 0.1

In [62]:
for epoch in range(epochs):
    error = 0
    for x, y in zip(X, Y):
        # Forward pass
        output = x
        for layer in network:
            output = layer.forward(output)
        
        # Calculate error
        error += mse(y, output)

        # Backward pass
        grad = mse_prime(y, output)
        
        for layer in reversed(network):
            grad = layer.backward(grad, learning_rate)
        
    error /= len(X)
    print(f"Epoch {epoch}, error: {error}")

Epoch 0, error: 0.7555841916425035
Epoch 1, error: 0.49772319533013376
Epoch 2, error: 0.49767922160639305
Epoch 3, error: 0.4976357577466576
Epoch 4, error: 0.49759266450211725
Epoch 5, error: 0.4975498168158711
Epoch 6, error: 0.49750710144938065
Epoch 7, error: 0.4974644150174584
Epoch 8, error: 0.4974216623489794
Epoch 9, error: 0.4973787551088119
Epoch 10, error: 0.4973356106303195
Epoch 11, error: 0.49729215091835355
Epoch 12, error: 0.49724830179078205
Epoch 13, error: 0.4972039921328839
Epoch 14, error: 0.49715915324383586
Epoch 15, error: 0.4971137182583505
Epoch 16, error: 0.49706762162954343
Epoch 17, error: 0.4970207986614892
Epoch 18, error: 0.49697318508181665
Epoch 19, error: 0.49692471664619703
Epoch 20, error: 0.4968753287677662
Epoch 21, error: 0.4968249561654742
Epoch 22, error: 0.4967735325260951
Epoch 23, error: 0.49672099017522064
Epoch 24, error: 0.49666725975300474
Epoch 25, error: 0.49661226989076723
Epoch 26, error: 0.4965559468848036
Epoch 27, error: 0.496498

# MNIST Problem

In [63]:
from keras.datasets import mnist
from keras.utils import np_utils

In [64]:
def preprocess_data(x, y, limit):
    zero_index = np.where(y == 0)[0][:limit]
    one_index = np.where(y == 1)[0][:limit]
    all_indices = np.hstack((zero_index, one_index))
    all_indices = np.random.permutation(all_indices)
    x, y = x[all_indices], y[all_indices]
    x = x.reshape(len(x), 1, 28, 28)
    x = x.astype('float32') / 255
    y = np_utils.to_categorical(y)
    y = y.reshape(len(y), 2, 1)
    return x, y

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, y_train = preprocess_data(x_train, y_train, 100)
x_test, y_test = preprocess_data(x_test, y_test, 100)

In [65]:
network = [
    Convolutional((1, 28, 28), 3, 5),
    Sigmoid(),
    Reshape((5, 26, 26), (5 * 26 * 26, 1)),
    Dense(5 * 26 * 26, 100),
    Sigmoid(),
    Dense(100, 2),
    Sigmoid()
]

epochs = 20
learning_rate = 0.1

for epoch in range(epochs):
    error = 0
    for x, y in zip(x_train, y_train):
        # Forward pass
        output = x
        for layer in network:
            output = layer.forward(output)
        
        # Calculate error
        error += binary_cross_entropy(y, output)

        # Backward pass
        grad = binary_cross_entropy_prime(y, output)
        
        for layer in reversed(network):
            grad = layer.backward(grad, learning_rate)
    
    error /= len(x_train)
    print(f"Epoch {epoch}, error: {error}")

for x, y in zip(x_test, y_test):
    output = x
    for layer in network:
        output = layer.forward(output)
    print(f"Predicted: {np.argmax(output)}, actual: {np.argmax(y)}")

Epoch 0, error: 0.49908735506804414
Epoch 1, error: 0.11089731682297345
Epoch 2, error: 0.1116519993659706
Epoch 3, error: 0.07293787906334981
Epoch 4, error: 0.08232771307962337
Epoch 5, error: 0.05431223724446682
Epoch 6, error: 0.02281582259986902
Epoch 7, error: 0.015719594503707027
Epoch 8, error: 0.01233767528776382
Epoch 9, error: 0.0115240846108301
Epoch 10, error: 0.009276726912901963
Epoch 11, error: 0.00836569561678335
Epoch 12, error: 0.008658310073350821
Epoch 13, error: 0.006746420111762868
Epoch 14, error: 0.0064402501675140285
Epoch 15, error: 0.006203151255724154
Epoch 16, error: 0.005517678010644039
Epoch 17, error: 0.004707781029864922
Epoch 18, error: 0.004207246777472628
Epoch 19, error: 0.003609174446689127
Predicted: 0, actual: 0
Predicted: 0, actual: 0
Predicted: 0, actual: 0
Predicted: 0, actual: 0
Predicted: 0, actual: 0
Predicted: 1, actual: 1
Predicted: 0, actual: 0
Predicted: 0, actual: 0
Predicted: 1, actual: 1
Predicted: 0, actual: 0
Predicted: 1, actual: